In [234]:
import config
import pandas as pd
from sodapy import Socrata
import mysql.connector
import alc_connect as alc


## Pulling All Arrest Data From Jan 1st 2019 - Present in Brooklyn, NY

In [5]:
client = Socrata(config.url, config.api_key)

results = client.get("uip8-fykc", arrest_boro="K", limit=50000)

df = pd.DataFrame.from_records(results)

In [6]:
df.head()

,arrest_key,arrest_date,pd_cd,pd_desc,ky_cd,ofns_desc,law_code,law_cat_cd,arrest_boro,arrest_precinct,jurisdiction_code,age_group,perp_sex,perp_race,x_coord_cd,y_coord_cd,latitude,longitude
0,203066328,2019-09-30T00:00:00.000,779,"PUBLIC ADMINISTRATION,UNCLASSI",126.0,MISCELLANEOUS PENAL LAW,PL 215510B,F,K,63,0,45-64,M,BLACK,1000521,168264,40.62851560000007,-73.94138369799998
1,203068778,2019-09-30T00:00:00.000,922,"TRAFFIC,UNCLASSIFIED MISDEMEAN",348.0,VEHICLE AND TRAFFIC LAWS,VTL0511001,M,K,70,0,45-64,M,BLACK,995070,176121,40.65008972100002,-73.96100881399997
2,203074208,2019-09-30T00:00:00.000,779,"PUBLIC ADMINISTRATION,UNCLASSI",126.0,MISCELLANEOUS PENAL LAW,PL 215510D,F,K,67,0,25-44,M,BLACK,1003705,176766,40.65184534600007,-73.92988853999998
3,203074210,2019-09-30T00:00:00.000,101,ASSAULT 3,344.0,ASSAULT 3 & RELATED OFFENSES,PL 1200001,M,K,84,0,18-24,F,BLACK,986229,193497,40.697789330000035,-73.99286451299997
4,203075937,2019-09-30T00:00:00.000,179,AGGRAVATED SEXUAL ASBUSE,116.0,SEX CRIMES,PL 130651A,F,K,73,0,18-24,M,BLACK,1008276,183623,40.670655072000045,-73.91339091999998


## Droping Irrelevant Columns

In [7]:
df.drop(['pd_cd', 'ky_cd', 'x_coord_cd','y_coord_cd','jurisdiction_code', 'law_code', 'arrest_key','arrest_precinct','arrest_boro', 'pd_desc'], axis = 1, inplace = True)

## Changing Object Type (Latitude, Longitude, Date)

In [8]:
df["latitude"] = df.latitude.astype(float)

In [9]:
df['longitude'] = df.longitude.astype(float)

In [10]:
df['arrest_date'] = pd.to_datetime(df['arrest_date'])

In [11]:
df.head()

,arrest_date,ofns_desc,law_cat_cd,age_group,perp_sex,perp_race,latitude,longitude
0,2019-09-30,MISCELLANEOUS PENAL LAW,F,45-64,M,BLACK,40.628516,-73.941384
1,2019-09-30,VEHICLE AND TRAFFIC LAWS,M,45-64,M,BLACK,40.650090,-73.961009
2,2019-09-30,MISCELLANEOUS PENAL LAW,F,25-44,M,BLACK,40.651845,-73.929889
3,2019-09-30,ASSAULT 3 & RELATED OFFENSES,M,18-24,F,BLACK,40.697789,-73.992865
4,2019-09-30,SEX CRIMES,F,18-24,M,BLACK,40.670655,-73.913391


## Renaming Columns

In [12]:
df.rename(columns={'arrest_date': 'Date', 'ofns_desc':'Crime_Description', 'law_cat_cd':'Offense_Level', 'age_group':'Age_Group','perp_sex':'Sex','perp_race':'Race','latitude':'Latitude','longitude':'Longitude'}, inplace=True)

In [77]:
df

,Date,Crime_Description,Offense_Level,Age_Group,Sex,Race,Latitude,Longitude
0,2019-09-30,MISCELLANEOUS PENAL LAW,F,45-64,M,BLACK,40.628516,-73.941384
1,2019-09-30,VEHICLE AND TRAFFIC LAWS,M,45-64,M,BLACK,40.650090,-73.961009
2,2019-09-30,MISCELLANEOUS PENAL LAW,F,25-44,M,BLACK,40.651845,-73.929889
3,2019-09-30,ASSAULT 3 & RELATED OFFENSES,M,18-24,F,BLACK,40.697789,-73.992865
4,2019-09-30,SEX CRIMES,F,18-24,M,BLACK,40.670655,-73.913391
...,...,...,...,...,...,...,...,...
45668,2019-01-01,PETIT LARCENY,M,25-44,M,WHITE,40.609900,-73.922373
45669,2019-01-01,DANGEROUS WEAPONS,F,25-44,M,BLACK,40.664828,-73.888304
45670,2019-01-01,DANGEROUS WEAPONS,F,25-44,M,BLACK,40.676052,-73.934611
45671,2019-01-01,FORGERY,F,45-64,M,BLACK,40.658586,-73.890624


In [225]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45673 entries, 0 to 45672
Data columns (total 9 columns):
Date                 45673 non-null datetime64[ns]
Crime_Description    45673 non-null object
Offense_Level        45673 non-null object
Age_Group            45673 non-null object
Sex                  45673 non-null object
Race                 45673 non-null object
Latitude             45673 non-null float64
Longitude            45673 non-null float64
average              45673 non-null float64
dtypes: datetime64[ns](1), float64(3), object(5)
memory usage: 3.1+ MB


## Droping Rows with Null Values (Col. Offense_Level & Crime_Description)

In [26]:
df = df.dropna(how='any',axis=0) 

In [27]:
df.count()

Date                 45673
Crime_Description    45673
Offense_Level        45673
Age_Group            45673
Sex                  45673
Race                 45673
Latitude             45673
Longitude            45673
dtype: int64

## Sending to SQL Database

In [240]:
df.to_sql('brooklyn_arrest', alc.db, if_exists = 'replace', index = False)

## Creating New DataFrames

In [148]:
by_crime = df.groupby(['Date','Crime_Description']).count()

In [217]:
by_crime

,Date,Crime_Description,Count
0,2019-01-01,ASSAULT 3 & RELATED OFFENSES,22
1,2019-01-01,CRIMINAL MISCHIEF & RELATED OF,5
2,2019-01-01,CRIMINAL TRESPASS,2
3,2019-01-01,DANGEROUS DRUGS,13
4,2019-01-01,DANGEROUS WEAPONS,9
...,...,...,...
7449,2019-09-30,RAPE,1
7450,2019-09-30,ROBBERY,14
7451,2019-09-30,SEX CRIMES,4
7452,2019-09-30,UNAUTHORIZED USE OF A VEHICLE,2


In [151]:
by_crime.drop(['Age_Group', 'Sex', 'Race','Latitude','Longitude'], axis = 1, inplace = True)

In [189]:
by_crime.rename(columns={'Offense_Level': 'Count'}, inplace=True)

In [200]:
by_crime.reset_index(inplace = True)

In [202]:
by_crime.set_index('Date')

,Crime_Description,Count
Date,,
2019-01-01,ASSAULT 3 & RELATED OFFENSES,22
2019-01-01,CRIMINAL MISCHIEF & RELATED OF,5
2019-01-01,CRIMINAL TRESPASS,2
2019-01-01,DANGEROUS DRUGS,13
2019-01-01,DANGEROUS WEAPONS,9
...,...,...
2019-09-30,RAPE,1
2019-09-30,ROBBERY,14
2019-09-30,SEX CRIMES,4


In [232]:
by_crime

,Date,Crime_Description,Count
0,2019-01-01,ASSAULT 3 & RELATED OFFENSES,22
1,2019-01-01,CRIMINAL MISCHIEF & RELATED OF,5
2,2019-01-01,CRIMINAL TRESPASS,2
3,2019-01-01,DANGEROUS DRUGS,13
4,2019-01-01,DANGEROUS WEAPONS,9
...,...,...,...
7449,2019-09-30,RAPE,1
7450,2019-09-30,ROBBERY,14
7451,2019-09-30,SEX CRIMES,4
7452,2019-09-30,UNAUTHORIZED USE OF A VEHICLE,2


In [222]:
df['Average_Temp'] = df[['Latitude', 'Longitude']].apply(lambda x: (x['Latitude'] + x['Longitude'])/2, axis = 1)

In [228]:
df.drop(['average'], axis = 1, inplace = True)

In [231]:
df.groupby(['Date', 'Sex']).count()

Crime_Description  Offense_Level  Age_Group  Race  Latitude  \
Date       Sex                                                                
2019-01-01 F                   18             18         18    18        18   
           M                  114            114        114   114       114   
2019-01-02 F                   41             41         41    41        41   
           M                  137            137        137   137       137   
2019-01-03 F                   27             27         27    27        27   
...                           ...            ...        ...   ...       ...   
2019-09-28 M                   90             90         90    90        90   
2019-09-29 F                   26             26         26    26        26   
           M                   77             77         77    77        77   
2019-09-30 F                   15             15         15    15        15   
           M                  103            103        103   103       103   

                Longitude  
Date       Sex             
2019-01-01 F           18  
           M          114  
2019-01-02 F           41  
           M          137  
2019-01-03 F           27  
...                   ...  
2019-09-28 M           90  
2019-09-29 F           26  
           M           77  
2019-09-30 F           15  
           M          103  

[546 rows x 6 columns]

In [241]:
all_data = pd.read_sql_query('Select * FROM brooklyn_weather JOIN brooklyn_arrest ON brooklyn_arrest.Date = brooklyn_weather.Date', alc.db)

dropping duplicate column ('Date')

In [287]:
cols = []
count = 1
for column in all_data.columns:
    if column == 'Date':
        cols.append(f'Date_{count}')
        count+=1
        continue
    cols.append(column)
all_data.columns = cols

In [291]:
all_data.drop(['Date_2'], axis = 1, inplace = True)

In [293]:
all_data.rename(columns ={'Date_1':'Date'}, inplace = True)

In [310]:
all_data.to_csv("output.csv")

In [306]:
dw = all_data.groupby(['Date', 'Forecast']).count()

dw - all_data with date and weather as multi_index

In [308]:
dw

,,Moon Phase,Wind Speed,Average_Temp,Crime_Description,Offense_Level,Age_Group,Sex,Race,Latitude,Longitude
Date,Forecast,,,,,,,,,,
2019-01-01,rain,132,132,132,132,132,132,132,132,132,132
2019-01-02,partly-cloudy-day,178,178,178,178,178,178,178,178,178,178
2019-01-03,partly-cloudy-day,226,226,226,226,226,226,226,226,226,226
2019-01-04,clear-day,205,205,205,205,205,205,205,205,205,205
2019-01-05,rain,175,175,175,175,175,175,175,175,175,175
...,...,...,...,...,...,...,...,...,...,...,...
2019-09-26,rain,184,184,184,184,184,184,184,184,184,184
2019-09-27,clear-day,168,168,168,168,168,168,168,168,168,168
2019-09-28,partly-cloudy-day,104,104,104,104,104,104,104,104,104,104
